# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,7.56,95,98,1.53,CL,1709521233
1,1,bredasdorp,-34.5322,20.0403,19.23,95,100,1.99,ZA,1709521234
2,2,talodi,10.6325,30.3797,25.43,12,36,3.11,SD,1709521234
3,3,adamstown,-25.0660,-130.1015,24.87,71,3,7.36,PN,1709521235
4,4,albany,42.6001,-73.9662,9.26,91,91,0.45,US,1709521147


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City"
    )

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df.loc[((city_data_df["Max Temp"] <= 30.0) & (city_data_df["Max Temp"] >= 20.0))\
                                 & (city_data_df["Cloudiness"] == 0),:]

# Drop any rows with null values
filtered_cities_df  = filtered_cities_df .dropna(how="any")

# Display sample data
filtered_cities_df .head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,44,pisco,-13.7000,-76.2167,24.03,78,0,6.69,PE,1709521256
94,94,tegucigalpa,14.0818,-87.2068,24.42,50,0,2.57,HN,1709521269
179,179,letlhakane,-21.4167,25.5833,20.02,48,0,3.88,BW,1709521302
198,198,aioun,16.6614,-9.6149,22.22,12,0,6.55,MR,1709521308
202,202,saint-louis,16.3333,-15.0000,22.44,17,0,4.96,SN,1709521309


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data 
hotel_df.head()

C:\Users\fiker\AppData\Local\Temp\ipykernel_42016\2475484846.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
44,pisco,PE,-13.7000,-76.2167,78,
94,tegucigalpa,HN,14.0818,-87.2068,50,
179,letlhakane,BW,-21.4167,25.5833,48,
198,aioun,MR,16.6614,-9.6149,12,
202,saint-louis,SN,16.3333,-15.0000,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = { "apiKey":geoapify_key,
    "limit":10,
    "categories":"accommodation.hotel"
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
     #hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho


,City,Country,Lat,Lng,Humidity,Hotel Name
44,pisco,PE,-13.7000,-76.2167,78,La Portada
94,tegucigalpa,HN,14.0818,-87.2068,50,Hotel Vizta Prado
179,letlhakane,BW,-21.4167,25.5833,48,Boteti Hotel
198,aioun,MR,16.6614,-9.6149,12,فندق العيون
202,saint-louis,SN,16.3333,-15.0000,17,No hotel found
225,juticalpa,HN,14.6500,-86.2000,55,Hotel La Hacienda
263,joal-fadiout,SN,14.1667,-16.8333,46,La lagune
305,sal rei,CV,16.1833,-22.9167,68,Hotel Marine Club
366,cam ranh,VN,11.9214,109.1591,58,Khách sạn Thái Bình
379,longboat key,US,27.4125,-82.6590,69,Zota Beach Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
    )

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)